In [1]:
!pip install pandas scikit-learn streamlit

  Using cached typing_extensions-4.15.0-py3-none-any.whl.metadata (3.3 kB)
   ---------------------------------------- 0.0/8.0 MB ? eta -:--:--
   ----------- ---------------------------- 2.4/8.0 MB 16.8 MB/s eta 0:00:01
   ---------------------------------------- 8.0/8.0 MB 29.3 MB/s  0:00:00
   ---------------------------------------- 0.0/9.0 MB ? eta -:--:--
   -------------------------------------- - 8.7/9.0 MB 44.9 MB/s eta 0:00:01
   ---------------------------------------- 9.0/9.0 MB 37.5 MB/s  0:00:00
   ---------------------------------------- 0.0/795.4 kB ? eta -:--:--
   ---------------------------------------- 795.4/795.4 kB 35.6 MB/s  0:00:00
   ---------------------------------------- 0.0/6.9 MB ? eta -:--:--
   ---------------------------------------- 6.9/6.9 MB 47.1 MB/s  0:00:00
Using cached typing_extensions-4.15.0-py3-none-any.whl (44 kB)
   ---------------------------------------- 0.0/28.0 MB ? eta -:--:--
   ----------------- ---------------------- 12.1/28.0 MB 58.

In [4]:
import pandas as pd
import re
import pickle

from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression


In [5]:
data = {
    "text": [
        "I love this product",
        "This is terrible",
        "It is okay, nothing special",
        "Very happy with the service!!",
        "I want a refund",
        "Support was slow",
        "Great experience overall"
    ],
    "label": [
        "positive",
        "negative",
        "neutral",
        "positive",
        "negative",
        "negative",
        "positive"
    ]
}

df = pd.DataFrame(data)
df


,text,label
0,I love this product,positive
1,This is terrible,negative
2,"It is okay, nothing special",neutral
3,Very happy with the service!!,positive
4,I want a refund,negative
5,Support was slow,negative
6,Great experience overall,positive


In [8]:
def preprocess(text: str) -> str:
    text = text.lower().strip()
    text = re.sub(r"http\S+", "<URL>", text)
    text = re.sub(r"\s+", " ", text)
    return text


In [9]:
df["clean_text"] = df["text"].apply(preprocess)
df


,text,label,clean_text
0,I love this product,positive,i love this product
1,This is terrible,negative,this is terrible
2,"It is okay, nothing special",neutral,"it is okay, nothing special"
3,Very happy with the service!!,positive,very happy with the service!!
4,I want a refund,negative,i want a refund
5,Support was slow,negative,support was slow
6,Great experience overall,positive,great experience overall


In [10]:
X = df["clean_text"]
y = df["label"]


In [11]:
pipeline = Pipeline([
    ("tfidf", TfidfVectorizer(ngram_range=(1, 2))),
    ("clf", LogisticRegression(max_iter=200))
])


In [12]:
pipeline.fit(X, y)


,"steps steps: list of tuplesList of (name of step, estimator) tuples that are to be chained insequential order. To be compatible with the scikit-learn API, all stepsmust define `fit`. All non-last steps must also define `transform`. See:ref:`Combining Estimators ` for more details.","[('tfidf', ...), ('clf', ...)]"
,"transform_input transform_input: list of str, default=NoneThe names of the :term:`metadata` parameters that should be transformed by thepipeline before passing it to the step consuming it.This enables transforming some input arguments to ``fit`` (other than ``X``)to be transformed by the steps of the pipeline up to the step which requiresthem. Requirement is defined via :ref:`metadata routing `.For instance, this can be used to pass a validation set through the pipeline.You can only set this if metadata routing is enabled, which youcan enable using ``sklearn.set_config(enable_metadata_routing=True)``... versionadded:: 1.6",None
,"memory memory: str or object with the joblib.Memory interface, default=NoneUsed to cache the fitted transformers of the pipeline. The last stepwill never be cached, even if it is a transformer. By default, nocaching is performed. If a string is given, it is the path to thecaching directory. Enabling caching triggers a clone of the transformersbefore fitting. Therefore, the transformer instance given to thepipeline cannot be inspected directly. Use the attribute ``named_steps``or ``steps`` to inspect estimators within the pipeline. Caching thetransformers is advantageous when fitting is time consuming. See:ref:`sphx_glr_auto_examples_neighbors_plot_caching_nearest_neighbors.py`for an example on how to enable caching.",None
,"verbose verbose: bool, default=FalseIf True, the time elapsed while fitting each step will be printed as itis completed.",False
,"input input: {'filename', 'file', 'content'}, default='content'- If `'filename'`, the sequence passed as an argument to fit is expected to be a list of filenames that need reading to fetch the raw content to analyze.- If `'file'`, the sequence items must have a 'read' method (file-like object) that is called to fetch the bytes in memory.- If `'content'`, the input is expected to be a sequence of items that can be of type string or byte.",'content'
,"encoding encoding: str, default='utf-8'If bytes or files are given to analyze, this encoding is used todecode.",'utf-8'
,"decode_error decode_error: {'strict', 'ignore', 'replace'}, default='strict'Instruction on what to do if a byte sequence is given to analyze thatcontains characters not of the given `encoding`. By default, it is'strict', meaning that a UnicodeDecodeError will be raised. Othervalues are 'ignore' and 'replace'.",'strict'
,"strip_accents strip_accents: {'ascii', 'unicode'} or callable, default=NoneRemove accents and perform other character normalizationduring the preprocessing step.'ascii' is a fast method that only works on characters that havea direct ASCII mapping.'unicode' is a slightly slower method that works on any characters.None (default) means no character normalization is performed.Both 'ascii' and 'unicode' use NFKD normalization from:func:`unicodedata.normalize`.",None
,"lowercase lowercase: bool, default=TrueConvert all characters to lowercase before tokenizing.",True
,"preprocessor preprocessor: callable, default=NoneOverride the preprocessing (string transformation) stage whilepreserving the tokenizing and n-grams generation steps.Only applies if ``analyzer`` is not callable.",None
,"tokenizer tokenizer: callable, default=NoneOverride the string tokenization step while preserving thepreprocessing and n-grams generation steps.Only applies if ``analyzer == 'word'``.",None


In [13]:
test_text = "Today was a freaky day"
clean_test = preprocess(test_text)

prediction = pipeline.predict([clean_test])[0]
confidence = max(pipeline.predict_proba([clean_test])[0])

prediction, round(confidence, 2)


('negative', 0.5)

In [14]:
probs = pipeline.predict_proba([clean_test])[0]
labels = pipeline.classes_

list(zip(labels, probs.round(2)))


[('negative', 0.5), ('neutral', 0.13), ('positive', 0.37)]

In [15]:
df_binary = df[df["label"] != "neutral"]

Xb = df_binary["clean_text"]
yb = df_binary["label"]

pipeline_binary = Pipeline([
    ("tfidf", TfidfVectorizer(ngram_range=(1,2))),
    ("clf", LogisticRegression(max_iter=200))
])

pipeline_binary.fit(Xb, yb)

pipeline_binary.predict_proba([clean_test])


array([[0.54473439, 0.45526561]])

In [18]:
import pickle

In [19]:
with open("model.pkl", "wb") as f:
    pickle.dump(pipeline, f)

print("Model saved successfully!")


Model saved successfully!


In [1]:
import os
os.getcwd()


'C:\\Users\\Dhiraj\\Desktop\\tiny_text_pipeline'

In [21]:
with open("model.pkl", "rb") as f:
    loaded_model = pickle.load(f)

loaded_model.predict([preprocess("I love this product")])

array(['positive'], dtype=object)

In [2]:
os.getcwd()

'C:\\Users\\Dhiraj\\Desktop\\tiny_text_pipeline'